<a href="https://colab.research.google.com/github/jeffrey7777/scylla/blob/main/SQL_Programming_Subqueries_using_Select_Statements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# SQL Programming - Subqueries using Select Statements

## 1.0 Connecting to our Database

In [1]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

'Connected: @None'

## 1.1 Loading Data from CSV Files

Datasets used in this notebook:


1.   **World Countries**

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1XlabneVWhvbmpRUi_Wix2FR5uYAAp-ZY/view?usp=sharing))

2.   **World Cities** 

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1R5952m-8hH91uiwDK8dcL9sT2D_PEcuw/view?usp=sharing))






In [2]:
# Importing Pandas
import pandas as pd

In [6]:
# Loading our Countries dataset
with open('world_countries.csv','r') as f:
    Countries = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS Countries;
%sql PERSIST Countries;
%sql SELECT * FROM Countries LIMIT 5;


 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


country,population,EU,coastline
Albania,2.9,no,yes
Andorra,0.07,no,no
Austria,8.57,yes,no
Belarus,9.48,no,no
Belgium,11.37,yes,yes


In [7]:
# Loading our cities dataset 

with open('world_cities.csv','r') as f:
    Cities = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS Cities;
%sql PERSIST Cities;
%sql SELECT * FROM Cities LIMIT 5;


 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


city,country,latitude,longitude,temperature
Aalborg,Denmark,57.03,9.92,7.52
Aberdeen,United Kingdom,57.17,-2.08,8.1
Abisko,Sweden,63.35,18.83,0.2
Adana,Turkey,36.99,35.32,18.67
Albacete,Spain,39.0,-1.87,12.62


## 1.2 Subqueries using the SELECT statement

A subquery is a query within another query. The outer query is called as main query and inner query is called as subquery. The subquery must be enclosed with parenthesis and generally executes first, and its output is used to complete the query condition for the main or outer query. We will go through examples of how subqueries can be used to answers different types of questions then later, we will use our learnings to work on the challenges.

In [8]:
# Example 1
# Subqueries can occur in the select list of the containing query
# as shown below;
# Let's find all countries in the Countries table with no city in the Cities table
# 
%%sql
SELECT country
FROM Countries
WHERE NOT EXISTS (SELECT * FROM Cities
                  WHERE Cities.country = Countries.country) 


 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [9]:
# Example 2
# Find the European countries with cities experiencing temperature > 20?
# 
%%sql
SELECT country
FROM Countries
WHERE EU = 'yes'
AND EXISTS (SELECT * FROM Cities
            WHERE Cities.country = Countries.country
            AND temperature > 15)

 * sqlite://
Done.


country
Greece
Italy
Portugal
Spain


In [10]:
# Example 3: Method 1
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM Cities C1
WHERE NOT EXISTS (SELECT * FROM Cities C2
                  WHERE C2.longitude < C1.longitude)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [11]:
# Example 3: Method 2
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM Cities
WHERE longitude = (SELECT min(longitude) FROM Cities)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [12]:
# Example 4:
# How many cities are in the EU?
# 
%%sql
SELECT count()
FROM Cities
WHERE country IN (SELECT country FROM Countries WHERE EU = 'yes')

 * sqlite://
Done.


count()
150


### <font color="green">1.2 Challenges</font>

In [16]:
# Challenge 1
# Find number of countries that have a city with latitude > 1000 (start with country list)
# 
%%sql
SELECT country
FROM Countries
WHERE EXISTS (SELECT * FROM Cities
            WHERE Cities.country = Countries.country
            AND latitude > 1000)

 * sqlite://
Done.


country


In [18]:
# Challenge 2
# Find three western most cities returning both the cities and their longitude
#
%%sql
SELECT city, longitude
FROM Cities
ORDER BY longitude ASC 
LIMIT 3;

 * sqlite://
Done.


city,longitude
Lisbon,-9.14
Galway,-9.05
Vigo,-8.73


In [56]:
# Challenge 3
# Find cities with temperature is more than 50% higher than the average; 
# We will return the city, country, and temperature, ordered by descending temperature
# Hint: avg(temperature) * 1.5
# 
%%sql
select city, country, temperature
from Cities
group by temperature
having (temperature) > (select 1.5 * avg(temperature) from Cities)

 * sqlite://
Done.


city,country,temperature
Marseille,France,14.98
Murcia,Spain,15.0
Denizli,Turkey,15.02
Catania,Italy,15.04
Rome,Italy,15.1
Pescara,Italy,15.13
Bari,Italy,15.15
Elbasan,Albania,15.18
Lisbon,Portugal,15.52
Badajoz,Spain,15.61


In [58]:
# Challenge 4
# How many cities are not in the EU and without a coastline?
# Hint: "not in"
# 
%%sql
SELECT count()
FROM Cities
WHERE country NOT IN (SELECT country FROM Countries WHERE EU = 'Yes' AND coastline = 'yes')

 * sqlite://
Done.


count()
213


In [59]:
# Challenge 5
# Find the number of countries with no coastline and a city with longitude < 10
# 
%%sql
select count()
from Countries
where coastline = 'no'
and exists (select * from Cities where country = Countries.country
            and longitude < 10)

 * sqlite://
Done.


count()
2


In [62]:
# Challenge 6
# Find countries in Countries table with no city in Cities table


%%sql
select country
from Countries
where not exists (select * from Cities
                  where Cities.country = Countries.country)

 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [64]:
# Challenge 7
# Find all cities in a country whose population is < 2, returning the city and country.
# 
%%sql
select city
from Cities
where exists (select * from Countries
                  where Cities.country = Countries.country and population < 2)

 * sqlite://
Done.


city
Andorra
Daugavpils
Podgorica
Riga
Tallinn
Tartu


In [65]:
# Challenge 8
# Find all countries with no city having a temperature > 6
# 
%%sql
select country
from Countries
where exists (select * from Cities
                  where Cities.country = Countries.country AND temperature<=6)

 * sqlite://
Done.


country
Austria
Belarus
Estonia
Finland
Germany
Latvia
Lithuania
Norway
Sweden
Turkey
